# Dataset

In [22]:
from sklearn.datasets import load_iris

# Carregando dataset de iris
dataset = load_iris()

# Guardando dados de entrada e as saídas desejadas para cada amostra de treinamento
amostras = dataset.data[:130]
y_desejado = dataset.target[:130]

# Parâmetros da rede neural

In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pyswarms as ps

# Arquiteura da rede neural
n_entrada = 4                # número de entradas na camada de entrada
n_oculta = 20                # número de neurônios na camada oculta
n_saida = 3                  # número de classes na camada de saídas
n_amostras = len(amostras)   # número de amostras

# Ativação, propagação e compração dos resultados

In [24]:
def ativacao1(p):
    # Construindo vetores iniciais de pesos e bias a partir da lista de parâmetros
    W1 = p[0:80].reshape((n_entrada,n_oculta))      # pesos oculta-entrada
    b1 = p[80:100].reshape((n_oculta,))             # bias oculta-entrada
    W2 = p[100:160].reshape((n_oculta,n_saida))     # pesos saída-oculta
    b2 = p[160:163].reshape((n_saida,))             # bias saída-oculta

    # Ativando camada oculta pela função de ativação sigmoid
    z1 = amostras.dot(W1) + b1      # nets da camada oculta
    a1 = np.tanh(z1)                # ativação da camada oculta
    y_obtido = a1.dot(W2) + b2      # nets da camada de saída
    return y_obtido                 # saídas da rede neural

# Propagação direta da rede neural
def forward(particula):
    y_obtido = ativacao1(particula)

    # Ativação da camada de saída pela função de softmax
    exp_scores = np.exp(y_obtido)
    probs = exp_scores / np.sum(exp_scores, axis=1, keepdims=True)

    # Calcula o erro da saída de rede
    corect_logprobs = -np.log(probs[range(n_amostras), y_desejado]) # erro por saída
    erro = np.sum(corect_logprobs) / n_amostras                     # erro médio total

    return erro

# Calculando a acurácia
def avaliacao(posicao):
    y_obtido = ativacao1(posicao)
    resultado = np.argmax(y_obtido, axis=1)
    return resultado

# Execução do PSO para otimização

In [25]:
import pso

def funcao_obj(enxame):
    n_particulas = enxame.shape[0]
    enxame = [forward(enxame[i]) for i in range(n_particulas)]
    return np.array(enxame)

# Definindo fator de aceleração 1 e 2 e peso de inércia para o PSO
parametrosPSO = {'c1': 0.5, 'c2': 0.3, 'w':0.9}
dimensoes = (n_entrada * n_oculta) + (n_oculta * n_saida) + n_oculta + n_saida
n_particulas = 100

# Iniciando PSO
otimizacao = ps.single.GlobalBestPSO(n_particles=n_particulas, dimensions=dimensoes, options=parametrosPSO)

# Realizando treinamento/otimização
custo, posicao = otimizacao.optimize(funcao_obj, iters=100, verbose=False)

'''otimizacao = pso.Pso(100, 100)
lb = np.array([-1] * dimensoes)
ub = np.array([1] * dimensoes)
posicao, custo = otimizacao.run(avaliacao, lb, ub)'''


'otimizacao = pso.Pso(100, 100)\nlb = np.array([-1] * dimensoes)\nub = np.array([1] * dimensoes)\nposicao, custo = otimizacao.run(avaliacao, lb, ub)'

# Resultado do treinamento

In [26]:
print("Valor ótimo de minização: ", custo)
print("Pesos otimizados da camada oculta:\n", posicao[:80])
print("Bias otimizados da camada oculta:\n", posicao[80:100])
print("Pesos otimizados da camada de saída:\n", posicao[100:160])
print("Bias otimizados da camada de saída:\n", posicao[160:])

Valor ótimo de minização:  0.07570331835663617
Pesos otimizados da camada oculta:
 [ 0.44478618 -1.3858267   1.8488291   0.33190313  1.10599251 -0.04388107
  0.50994891  3.8343367   0.75186585 -0.74486942  0.38504302  1.30185415
 -2.35067362  0.12810108 -0.60181683  0.746093    1.71235643  2.53219746
 -1.16565544 -1.70832813 -0.07769204  0.69811644 -0.49545361 -0.90762233
  1.21788242  0.77492863 -0.4384671   0.17080024  1.12286987 -2.27169875
  0.7216708   1.74292733  0.26570319  0.12406611 -0.40649683  1.08586408
  0.7906285  -4.02562785  0.41811955  1.83592224  0.22592007 -1.5387133
  0.7518746   1.0744207  -2.16067381  0.02571934  0.95890122  1.23602889
  1.24124227  1.34448362  0.53579251 -0.5250248  -1.05787644  0.64620142
  0.23662815 -2.21265728  0.01866982 -0.10675471 -0.09516524  1.77001776
 -0.73809639  0.38054598  1.1110047   1.3170216  -0.48603002 -0.4729372
  0.39345552  1.15650984 -1.93615832  2.71429933  1.75636613 -1.81680224
  0.1026108   1.18177642  1.35130348 -0.031

# Avaliação do treinamento e do teste

In [27]:
# Acurácia do treino
print("Acurácia de treino: ", (np.mean(avaliacao(posicao) == y_desejado) * 100))

Acurácia de treino:  96.92307692307692


# Execução e avaliação do teste

In [28]:
# Guardando dados de entrada e as saídas desejadas para cada amostra de teste
amostras = dataset.data[130:]
y_desejado = dataset.target[130:]

# Acurácia do teste
print("Acurácia de teste: ", (np.mean(avaliacao(posicao) == y_desejado) * 100))

Acurácia de teste:  90.0
